In [5]:
import json
import pendulum
import logging
import xml.etree.ElementTree as ET
import csv
# from airflow.decorators import (
#     dag,
#     task,
# ) 
import requests
# from pymongo import MongoClient
import os


default_args = {
    "owner": "riyaz",
    "retries": 3
}


def extract():
    res = requests.get("https://timesofindia.indiatimes.com/rssfeedstopstories.cms")
    if not os.path.exists("raw"):
        os.mkdir("raw")
    filename = f"raw/raw_rss_feed_{pendulum.now()}.xml"
    with open(filename, "w") as file:
        file.write(res.text) 
    return filename

In [8]:
extracted_filename=extract()

In [9]:
extracted_filename

'raw/raw_rss_feed_2024-09-12 06:23:27.577179+07:00.xml'

In [7]:
def transform(filename):
    """
    Memproses file XML dan mengonversinya menjadi file CSV.
    """
    print(f"Transforming data from {filename}...")
    tree = ET.parse(filename)
    root = tree.getroot()
    
    # Ambil item dari XML
    items = []
    for item in root.findall('.//item'):
        title = item.find('title').text
        description = item.find('description').text
        guid = item.find("guid").text
        items.append((title, description, guid))
    
    # Buat direktori "curated" jika belum ada
    if not os.path.exists("curated"):
        os.mkdir("curated")
    
    # Simpan hasil kurasi dalam file CSV
    curated_filename = f'curated/curated_{pendulum.now().to_iso8601_string()}.csv'
    with open(curated_filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Title', 'Description', 'GUID'])
        writer.writerows(items)
    
    print(f"Data transformed and saved to {curated_filename}")
    return curated_filename


In [10]:
transform(extracted_filename)

Transforming data from raw/raw_rss_feed_2024-09-12 06:23:27.577179+07:00.xml...
Data transformed and saved to curated/curated_2024-09-12T06:23:39.627831+07:00.csv


'curated/curated_2024-09-12T06:23:39.627831+07:00.csv'